# Convolutional Neural Network Group Project

## Mask Detection Using CNN

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

## Data Preprocessing

### Function to tell which image is corrupted

In [2]:
def isCorrupted(fileimage):
    try:
        with Image.open(fileimage) as img:
            img.verify() # verify that it is, in fact an image
        return False
    except:
        return True

### Removing corrupted images

In [3]:
import os
for dirname, _, filenames in os.walk('Face Mask Dataset/'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if(not isCorrupted(os.path.join(dirname, filename))):
            os.remove(os.path.join(dirname, filename))
            print("File removed as it was corrupted\t",os.path.join(dirname, filename))


### Function to change image to grey scale for easier processing

In [24]:
import cv2
def convertImageToGray(imageFeed):
    image = cv2.imread(imageFeed)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filename = f"{os.getpid()}.png"
    cv2.imwrite(filename, gray)
    print('The image is converted to Grayscale successfully')

### Preprocessing the Training set

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Face Mask Dataset/Train',
                                                 target_size = (64, 64),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 10992 images belonging to 2 classes.


### Preprocessing the Test set

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('Face Mask Dataset/Test',
                                            target_size = (64, 64),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 992 images belonging to 2 classes.


## Building the CNN

### Initialising the CNN

In [7]:
model = tf.keras.models.Sequential()

### Convolution

In [8]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Pooling

In [9]:
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [10]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening

In [11]:
model.add(tf.keras.layers.Flatten())

### Full Connection

In [12]:
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Output Layer

In [13]:
model.add(tf.keras.layers.Dense(units=2, activation='softmax'))

### Compiling the CNN Model

In [14]:
model.compile(optimizer = 'Adagrad', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
model.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
687/687 [==============================] - 65s 94ms/step - loss: 0.4160 - accuracy: 0.8368 - val_loss: 0.2810 - val_accuracy: 0.9012
Epoch 2/10
687/687 [==============================] - 70s 102ms/step - loss: 0.2619 - accuracy: 0.9085 - val_loss: 0.2326 - val_accuracy: 0.9214
Epoch 3/10
687/687 [==============================] - 63s 92ms/step - loss: 0.2190 - accuracy: 0.9233 - val_loss: 0.1956 - val_accuracy: 0.9274
Epoch 4/10
687/687 [==============================] - 58s 85ms/step - loss: 0.1922 - accuracy: 0.9310 - val_loss: 0.1713 - val_accuracy: 0.9415
Epoch 5/10
687/687 [==============================] - 67s 98ms/step - loss: 0.1694 - accuracy: 0.9379 - val_loss: 0.1547 - val_accuracy: 0.9466
Epoch 6/10
687/687 [==============================] - 74s 108ms/step - loss: 0.1558 - accuracy: 0.9427 - val_loss: 0.1471 - val_accuracy: 0.9506
Epoch 7/10
687/687 [==============================] - 69s 100ms/step - loss: 0.1420 - accuracy: 0.9490 - val_loss: 0.1319 - val_accura

## Part 4 - Making a single prediction

In [16]:
import numpy as np
import tensorflow as tf
test_image = tf.keras.utils.load_img('Face Mask Dataset/OutsideTestData/3.png', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
print(result)
x=result[0][0]
y=result[0][1]
if(x>y):
    prediction="Mask Detected"
else:
    prediction="No Mask Detected"

1/1 [==============================] - 0s 121ms/step
[[0. 1.]]


In [17]:
print(prediction)

No Mask Detected


In [18]:
import numpy as np
import tensorflow as tf
test_image = tf.keras.utils.load_img('Face Mask Dataset/OutsideTestData/1.png', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
print(result)
x=result[0][0]
y=result[0][1]
if(x>y):
    prediction="Mask Detected"
else:
    prediction="No Mask Detected"

1/1 [==============================] - 0s 23ms/step
[[1. 0.]]


In [19]:
print(prediction)

Mask Detected


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

### Trying to save a model

In [21]:
model_json=model.to_json()
with open( "model.json" , "w" ) as f1:
    f1.write(model_json)
# seríalize weíghts to HDr5
model.save_weights("model.h5")

### load json and create model

In [22]:
from keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")

### Loaded Model testing if working properly

In [23]:
import numpy as np
import tensorflow as tf
test_image = tf.keras.utils.load_img('Face Mask Dataset/OutsideTestData/6.png', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
if result[0][0] ==1:
  prediction = 'No Mask Detected'
else:
  prediction = 'Mask Detected'
print(prediction)

1/1 [==============================] - 0s 65ms/step
No Mask Detected


### -Made by Tarun Kumar